In [ ]:
# Import necessary libraries
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env
import torch
from envs.aquaculture_env import AquacultureEnv
from utils.plot_callback import PlotCallback


# Discretized environment to handle continuous action space
class DiscretizedAquacultureEnv(Env):
    def __init__(self, region="guangdong"):
        self.base_env = AquacultureEnv(region=region)
        
        # Discretize the action space
        self.feed_bins = 40
        self.temp_bins = 16
        self.air_bins = 10

        self.discrete_actions = [
            (feed, temp, air)
            for feed in np.linspace(self.base_env.action_space.low[0], self.base_env.action_space.high[0], self.feed_bins)
            for temp in np.linspace(self.base_env.action_space.low[1], self.base_env.action_space.high[1], self.temp_bins)
            for air in np.linspace(self.base_env.action_space.low[2], self.base_env.action_space.high[2], self.air_bins)
        ]
        
        self.action_space = Discrete(len(self.discrete_actions))
        self.observation_space = self.base_env.observation_space

    def reset(self, **kwargs):
        obs, info = self.base_env.reset(**kwargs)
        return obs, info

    def step(self, action_idx):
        action = np.array(self.discrete_actions[action_idx], dtype=np.float32)
        obs, reward, terminated, truncated, info = self.base_env.step(action)
        return obs, reward, terminated, truncated, info

    def render(self, mode='human'):
        return self.base_env.render()

    def close(self):
        self.base_env.close()


In [2]:
# Check if CUDA is available for model training
print(torch.cuda.is_available())

# Create the environment and validate it
env = DiscretizedAquacultureEnv(region="north_sulawesi")
check_env(env, warn=True)

# Initialize the DQN model
for eps in [0.05, 0.01, 0.001]:
    plot_cb = PlotCallback(
        window=10,
        save_path=f"training_rewards_eps_{eps}.png",
        title=f"Training Rewards (Final ε={eps})"
    )

    model = DQN(
        "MlpPolicy",
        env,
        verbose=1,
        tensorboard_log=f"./aqua_tensorboard_dqn_eps_{eps}",
        learning_rate=1e-4,
        batch_size=256,
        gamma=0.99,
        exploration_final_eps=eps,
        device="cuda"
    )
    print(f"\n🚀 Training with exploration_final_eps = {eps}")
    model.learn(total_timesteps=180 * 10, callback=plot_cb)

False
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

🚀 Training with exploration_final_eps = 0.05
Logging to ./aqua_tensorboard_dqn_eps_0.05\DQN_4
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 180      |
|    ep_rew_mean      | -18.1    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 419      |
|    time_elapsed     | 1        |
|    total_timesteps  | 720      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.252    |
|    n_updates        | 154      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 180      |
|    ep_rew_mean      | -13.6    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 423      |


In [8]:
import optuna
import os
from datetime import datetime
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_checker import check_env
from utils.plot_callback import PlotCallback

PLOT_DIR = "plots"
os.makedirs(PLOT_DIR, exist_ok=True)
RUN_ID   = datetime.now().strftime("%Y%m%d_%H%M%S")

eval_env = DiscretizedAquacultureEnv(region="north_sulawesi")
eval_cb  = EvalCallback(
    eval_env,
    n_eval_episodes=20,
    eval_freq=180 * 5,
    deterministic=True
)

def objective(trial: optuna.Trial) -> float:
    lr     = trial.suggest_float("learning_rate", 1e-4, 5e-4, log=True)
    gamma  = trial.suggest_float("gamma",           0.98,  0.995)
    batch  = trial.suggest_categorical("batch_size", [32, 64])
    buf    = trial.suggest_categorical("buffer_size", [500_000, 1000_000])
    tgt_i  = trial.suggest_categorical("target_update_interval", [5000, 10000])
    net    = trial.suggest_categorical(
                "net_arch",
                [[64, 64], [128, 128]]
             )
    env = DiscretizedAquacultureEnv(region="north_sulawesi")

    model = DQN(
        "MlpPolicy",
        env,
        learning_rate=lr,
        gamma=gamma,
        batch_size=batch,
        buffer_size=buf,
        target_update_interval=tgt_i,
        exploration_final_eps=0.01,
        policy_kwargs=dict(net_arch=net),
        verbose=0,
        tensorboard_log="./aqua_tensorboard",
        device="cuda" if torch.cuda.is_available() else "cpu",
    )

    plot_cb = PlotCallback(
        window=1,
        save_path=os.path.join(PLOT_DIR, f"{RUN_ID}_trial{trial.number:02d}_rewards.png"),
        title=f"DQN Trial {trial.number}"
    )

    model.learn(180 * 100, callback=[eval_cb, plot_cb])
    return eval_cb.best_mean_reward

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=16, timeout=3 * 3600)

print("Best value:", study.best_value)
print("Best params:", study.best_params)

[I 2025-04-25 17:29:10,986] A new study created in memory with name: no-name-d305c081-5c76-40a2-a3bb-2cc59fc639f8
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64, 64] which is of type list.
  warnings.warn(message)
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128] which is of type list.
  warnings.warn(message)
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to

Eval num_timesteps=900, episode_reward=0.10 +/- 0.97
Episode length: 180.00 +/- 0.00
New best mean reward!
Eval num_timesteps=1800, episode_reward=0.01 +/- 0.99
Episode length: 180.00 +/- 0.00
Eval num_timesteps=2700, episode_reward=5.53 +/- 2.14
Episode length: 180.00 +/- 0.00
New best mean reward!
Eval num_timesteps=3600, episode_reward=58.48 +/- 1.45
Episode length: 180.00 +/- 0.00
New best mean reward!
Eval num_timesteps=4500, episode_reward=101.09 +/- 2.08
Episode length: 180.00 +/- 0.00
New best mean reward!
Eval num_timesteps=5400, episode_reward=101.60 +/- 1.54
Episode length: 180.00 +/- 0.00
New best mean reward!
Eval num_timesteps=6300, episode_reward=101.10 +/- 1.41
Episode length: 180.00 +/- 0.00
Eval num_timesteps=7200, episode_reward=55.85 +/- 2.27
Episode length: 180.00 +/- 0.00
Eval num_timesteps=8100, episode_reward=10.95 +/- 3.02
Episode length: 180.00 +/- 0.00
Eval num_timesteps=9000, episode_reward=-14.89 +/- 3.98
Episode length: 180.00 +/- 0.00
Eval num_timesteps=9

[I 2025-04-25 17:30:31,746] Trial 0 finished with value: 180.81518916226923 and parameters: {'learning_rate': 0.00010290358907837275, 'gamma': 0.9826490514542094, 'batch_size': 32, 'buffer_size': 1000000, 'target_update_interval': 5000, 'net_arch': [128, 128]}. Best is trial 0 with value: 180.81518916226923.


Eval num_timesteps=18000, episode_reward=38.78 +/- 3.93
Episode length: 180.00 +/- 0.00
✅ Training curve saved to: plots\20250425_172910_trial00_rewards.png
📊 Total reward: 7957.21
📉 Reward variation (std dev): 61.83
Eval num_timesteps=900, episode_reward=-105.45 +/- 2.48
Episode length: 180.00 +/- 0.00
Eval num_timesteps=1800, episode_reward=24.47 +/- 1.47
Episode length: 180.00 +/- 0.00
Eval num_timesteps=2700, episode_reward=32.06 +/- 1.31
Episode length: 180.00 +/- 0.00
Eval num_timesteps=3600, episode_reward=-83.21 +/- 1.93
Episode length: 180.00 +/- 0.00
Eval num_timesteps=4500, episode_reward=20.98 +/- 1.66
Episode length: 180.00 +/- 0.00
Eval num_timesteps=5400, episode_reward=-195.85 +/- 2.64
Episode length: 180.00 +/- 0.00
Eval num_timesteps=6300, episode_reward=15.62 +/- 1.92
Episode length: 180.00 +/- 0.00
Eval num_timesteps=7200, episode_reward=-21.02 +/- 3.24
Episode length: 180.00 +/- 0.00
Eval num_timesteps=8100, episode_reward=-1.31 +/- 1.99
Episode length: 180.00 +/- 

[I 2025-04-25 17:31:55,459] Trial 1 finished with value: 180.81518916226923 and parameters: {'learning_rate': 0.00013078368361118422, 'gamma': 0.9871982329503499, 'batch_size': 64, 'buffer_size': 500000, 'target_update_interval': 10000, 'net_arch': [128, 128]}. Best is trial 0 with value: 180.81518916226923.


...

Eval num_timesteps=18000, episode_reward=153.18 +/- 1.00
Episode length: 180.00 +/- 0.00
✅ Training curve saved to: plots\20250425_172910_trial13_rewards.png
📊 Total reward: 6865.00
📉 Reward variation (std dev): 77.75
Eval num_timesteps=900, episode_reward=-144.94 +/- 2.46
Episode length: 180.00 +/- 0.00
Eval num_timesteps=1800, episode_reward=-3.38 +/- 2.46
Episode length: 180.00 +/- 0.00
Eval num_timesteps=2700, episode_reward=-26.39 +/- 2.67
Episode length: 180.00 +/- 0.00
Eval num_timesteps=3600, episode_reward=-32.63 +/- 2.50
Episode length: 180.00 +/- 0.00
Eval num_timesteps=4500, episode_reward=-25.41 +/- 12.39
Episode length: 180.00 +/- 0.00
Eval num_timesteps=5400, episode_reward=20.18 +/- 1.46
Episode length: 180.00 +/- 0.00
Eval num_timesteps=6300, episode_reward=61.05 +/- 1.77
Episode length: 180.00 +/- 0.00
Eval num_timesteps=7200, episode_reward=86.04 +/- 2.24
Episode length: 180.00 +/- 0.00
Eval num_timesteps=8100, episode_reward=90.87 +/- 1.69
Episode length: 180.00 +/-

[I 2025-04-25 17:48:43,187] Trial 14 finished with value: 186.3059869751334 and parameters: {'learning_rate': 0.0004956895877833771, 'gamma': 0.9932509044902064, 'batch_size': 64, 'buffer_size': 1000000, 'target_update_interval': 5000, 'net_arch': [128, 128]}. Best is trial 2 with value: 186.3059869751334.


Eval num_timesteps=18000, episode_reward=160.45 +/- 0.64
Episode length: 180.00 +/- 0.00
✅ Training curve saved to: plots\20250425_172910_trial14_rewards.png
📊 Total reward: 9446.17
📉 Reward variation (std dev): 65.00
Eval num_timesteps=900, episode_reward=-193.32 +/- 2.96
Episode length: 180.00 +/- 0.00
Eval num_timesteps=1800, episode_reward=-21.93 +/- 1.94
Episode length: 180.00 +/- 0.00
Eval num_timesteps=2700, episode_reward=-23.80 +/- 21.15
Episode length: 180.00 +/- 0.00
Eval num_timesteps=3600, episode_reward=-84.01 +/- 3.07
Episode length: 180.00 +/- 0.00
Eval num_timesteps=4500, episode_reward=-12.12 +/- 1.76
Episode length: 180.00 +/- 0.00
Eval num_timesteps=5400, episode_reward=-1.07 +/- 1.62
Episode length: 180.00 +/- 0.00
Eval num_timesteps=6300, episode_reward=71.66 +/- 2.57
Episode length: 180.00 +/- 0.00
Eval num_timesteps=7200, episode_reward=41.82 +/- 4.58
Episode length: 180.00 +/- 0.00
Eval num_timesteps=8100, episode_reward=69.14 +/- 1.44
Episode length: 180.00 +/

[I 2025-04-25 17:49:55,209] Trial 15 finished with value: 186.3059869751334 and parameters: {'learning_rate': 0.00019076264179196379, 'gamma': 0.9893050542392028, 'batch_size': 32, 'buffer_size': 500000, 'target_update_interval': 5000, 'net_arch': [64, 64]}. Best is trial 2 with value: 186.3059869751334.


Eval num_timesteps=18000, episode_reward=-129.73 +/- 4.42
Episode length: 180.00 +/- 0.00
✅ Training curve saved to: plots\20250425_172910_trial15_rewards.png
📊 Total reward: 2862.38
📉 Reward variation (std dev): 87.29
Best value: 186.3059869751334
Best params: {'learning_rate': 0.00011556060083557823, 'gamma': 0.9949461829904909, 'batch_size': 64, 'buffer_size': 1000000, 'target_update_interval': 5000, 'net_arch': [128, 128]}


In [9]:
import torch
import numpy as np
import os
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env
from utils.plot_callback import PlotCallback

# Check CUDA and env
print("CUDA Available:", torch.cuda.is_available())
env = DiscretizedAquacultureEnv(region="north_sulawesi")
check_env(env, warn=True)

# Best hyperparameters from tuning
best_params = {
    "learning_rate": 1e-4,
    "gamma": 0.995,
    "batch_size": 64,
    "buffer_size": 1000000,
    "target_update_interval": 5000,
    "net_arch": [128, 128],
    "exploration_final_eps": 0.01
}

# Instantiate DQN with fixed final epsilon
model = DQN(
    "MlpPolicy",
    env,
    learning_rate=best_params["learning_rate"],
    gamma=best_params["gamma"],
    batch_size=best_params["batch_size"],
    buffer_size=best_params["buffer_size"],
    target_update_interval=best_params["target_update_interval"],
    exploration_final_eps=best_params["exploration_final_eps"],
    policy_kwargs=dict(net_arch=best_params["net_arch"]),
    verbose=1,
    tensorboard_log="./aqua_tensorboard",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Training callback
plot_cb = PlotCallback(
    window=1,
    save_path="plots/dqn_training_rewards.png",
    title=f"DQN Training Rewards (ε={best_params['exploration_final_eps']})"
)

# Train
model.learn(total_timesteps=180 * 300, callback=plot_cb)

# Save
model_save_path = "./saved_model/dqn_best_model"
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
model.save(model_save_path)
print(f"💾 Model saved to {model_save_path}")

CUDA Available: False
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./aqua_tensorboard\DQN_46
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 180      |
|    ep_rew_mean      | 16.8     |
|    exploration_rate | 0.868    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 555      |
|    time_elapsed     | 1        |
|    total_timesteps  | 720      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.202    |
|    n_updates        | 154      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 180      |
|    ep_rew_mean      | 13.9     |
|    exploration_rate | 0.736    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 539      |
|    time_elapsed     | 2        |
|    to

In [5]:
import time
from utils.calculation import Calculation

obs, _ = env.reset()
terminated = False
truncated = False
total_reward = 0

print("Discrete action space size:", env.action_space.n)

while not (terminated or truncated):
    action_idx, _ = model.predict(obs, deterministic=True)
    action = env.discrete_actions[action_idx]

    raw = env.base_env.denormalize(obs)
    biomass, fish_count, temp, do_level, uia = raw
    feed_amount = Calculation.compute_feed_weight(action[0], biomass)
    print(f"""\n--- Day {env.base_env.day + 1} ---
Raw-obs: biomass={biomass:.1f} g, count={fish_count:.0f}, temp={temp:.2f}°C, DO={do_level:.2f} mg/L, UIA={uia:.3f} mg/L
Action: feed_rate={action[0]:.3f} → feed_amt={feed_amount:.2f} g, temp_set={action[1]:.3f}, aeration_rate={action[2]:.2f}mg/L
""")

    obs, reward, terminated, truncated, info = env.step(action_idx)
    total_reward += reward

    print(f"""Reward Breakdown:
Fish Value Gain:     {info['fish_value']} $
Feed Cost:           {info['feed_cost']:.2f} $
Heat Cost:           {info['heat_cost']:.2f} $
Oxygenation Cost:    {info['oxygenation_cost']:.2f} $
→ Net Reward:        {info['reward']:.2f} $
""")

    env.render()

    if hasattr(env.base_env, 'exit_requested') and env.base_env.exit_requested:
        break

    time.sleep(0.01)

env.close()
print("Total Reward:", total_reward)

TypeError: AquacultureEnv.__init__() got an unexpected keyword argument 'discrete'